# Downloading the data

This notebook is used to download the ERA5 data from weatherbench2.


In [7]:
import xarray as xr
import os
import fsspec
import numpy as np
import pandas as pd
import jinja2

the following line should be copied into a command window to execute.
the first one is used to download 6hourly ERA5 data from 1959-2022 on a 64x32 resolution.
the second one downloads the ERA5 climatology, this is used during the evaluation.

In [ ]:
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr" ./ERA5_data
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr" ./ERA5_data

## transform data
in this section we will transform the downloaded data to work properly with the graphcast model.

In [8]:
# Define the relative path to the file
relative_path = "ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"
# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)
# Open the Zarr file using xarray
obs_data = xr.open_zarr(absolute_path)
obs_data

/home/jupyter-lukas/Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr


to reduce the file sizes we remove all variables we do not use in our model from the downloading data.

In [10]:
# prune the dataset of unnecessary variables.
for x in obs_data.data_vars:
    if x not in ['2m_temperature','mean_sea_level_pressure','10m_v_component_of_wind',
        '10m_u_component_of_wind','total_precipitation_6hr','toa_incident_solar_radiation',
        'temperature','geopotential','geopotential_at_surface','land_sea_mask',
        'u_component_of_wind','v_component_of_wind','vertical_velocity','specific_humidity']:
        obs_data = obs_data.drop_vars(x)

<xarray.Dataset>
Dimensions:                       (time: 92044, longitude: 64, latitude: 32,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 -87.19 -81.56 ... 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
Data variables: (12/14)
    10m_u_component_of_wind       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    geopotential                  (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    land_sea_mask                 (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

Graphcast requires the longitude and latitude dimensions to written in short form as 'lon' and 'lat'.
for doing evaluation this will need to be undone.

Graphcast also requires a 'datetime' coordinate even though this coord is not used in the model itself.
Graphcast also requries a 'batch' dimension equal to 1. so we add that as well the all vars.

In [11]:
#rename obs data:
obs_data = obs_data.rename({'longitude': 'lon'})
obs_data = obs_data.rename({'latitude': 'lat'})

# add datetime coord:
copied_coord = obs_data['time'].copy()
# Rename the copied coordinate to the new name
obs_data = obs_data.assign_coords(datetime=copied_coord)

# add batch dim:
obs_data = obs_data.expand_dims('batch')
obs_data['datetime'] = obs_data['datetime'].expand_dims('batch')

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
  * lat                           (lat) float64 -87.19 -81.56 ... 81.56 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * lon                           (lon) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
    datetime                      (batch, time) datetime64[ns] 1959-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>

In [13]:
# save changes made to file.
obs_data.to_zarr('/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr', mode='w')

In [14]:
#checking if file is saved properly and data is not corrupted
obs_data = xr.open_zarr('/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr')
obs_data['temperature'].values

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (batch, time) datetime64[ns] dask.array<chunksize=(1, 23011), meta=np.ndarray>
  * lat                           (lat) float64 -87.19 -81.56 ... 81.56 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * lon                           (lon) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>